In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import shutil
import torch
from pathlib import Path
from runner import Runner
import interest_evolution
# from full_slate_q_agent import FullSlateQAgent
# from ppo  import PPOAgentWrapper
from bandit import BanditAgentWrapper
from random_agent import RandomAgent

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
seed = 0
np.random.seed(seed)

env_config = {
    'num_candidates': 10,
    'slate_size': 2,
    'resample_documents': True,
    'seed': seed,
}

In [5]:
tmp_base_dir = './logs/random'

# Automatically delete train and eval directories if they exist
train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

# Recreate base log directory
Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)

def create_random_agent(env, **kwargs):
    return RandomAgent(action_space=env.action_space)
    
# --- Initialize training runner ---
runner_random = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_random_agent,
    env=interest_evolution.create_environment(env_config),
)

# --- Run training + evaluation ---
runner_random.run_training(max_training_steps=2000, num_iterations=100)
runner_random.run_evaluation(max_eval_episodes=5)

Training Episode:   2%|▏         | 2/100 [00:00<00:14,  6.57it/s]

[TRAIN][Step 2010] AvgLen: 35.26 | AvgRew: 74.30 | StdRew: 9.23 | Time/Step: 0.0001
[TRAIN][Step 4034] AvgLen: 34.90 | AvgRew: 74.98 | StdRew: 8.64 | Time/Step: 0.0001


Training Episode:   4%|▍         | 4/100 [00:00<00:14,  6.54it/s]

[TRAIN][Step 6072] AvgLen: 37.05 | AvgRew: 75.36 | StdRew: 7.20 | Time/Step: 0.0001
[TRAIN][Step 8095] AvgLen: 38.17 | AvgRew: 77.62 | StdRew: 11.40 | Time/Step: 0.0001


Training Episode:   6%|▌         | 6/100 [00:00<00:14,  6.61it/s]

...

[TRAIN][Step 195720] AvgLen: 35.75 | AvgRew: 75.47 | StdRew: 8.48 | Time/Step: 0.0001
[TRAIN][Step 197727] AvgLen: 36.49 | AvgRew: 75.38 | StdRew: 8.75 | Time/Step: 0.0001


Training Episode: 100%|██████████| 100/100 [00:15<00:00,  6.39it/s]

[TRAIN][Step 199741] AvgLen: 38.00 | AvgRew: 77.13 | StdRew: 7.96 | Time/Step: 0.0001
[TRAIN][Step 201745] AvgLen: 36.44 | AvgRew: 75.64 | StdRew: 7.64 | Time/Step: 0.0001
Training plot path:  ./logs/random/train/plot_data.json



Evaluation Episode: 100%|██████████| 5/5 [00:00<00:00, 366.54it/s]

[EVAL] CurrentAgent | Episode 1 | Reward: 64.79 | Length: 34
[EVAL] CurrentAgent | Episode 2 | Reward: 69.68 | Length: 37
[EVAL] CurrentAgent | Episode 3 | Reward: 71.66 | Length: 37
[EVAL] CurrentAgent | Episode 4 | Reward: 66.32 | Length: 30
[EVAL] CurrentAgent | Episode 5 | Reward: 68.00 | Length: 41
[EVAL][Step 0] AvgLen: 35.80 | AvgRew: 68.09 | StdRew: 2.42 | Time/Step: 0.0000
Plot path:  ./logs/random/train/eval_plot_data.json


In [6]:
# UCB_Bandit 
tmp_base_dir = './logs/UCB_bandit'
def create_bandit_agent(env):
    return BanditAgentWrapper(n_arms=env.action_space.nvec[0])


# Delete and recreate directories
train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')
for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)
Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)

# Run Bandit agent
runner_bandit = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_bandit_agent,
    env=interest_evolution.create_environment(env_config),
)

runner_bandit.run_training(max_training_steps=2000, num_iterations=50)
runner_bandit.run_evaluation(max_eval_episodes=5)

Training Episode:   4%|▍         | 2/50 [00:00<00:07,  6.40it/s]

[TRAIN][Step 2024] AvgLen: 47.07 | AvgRew: 68.83 | StdRew: 8.86 | Time/Step: 0.0001
[TRAIN][Step 4046] AvgLen: 53.21 | AvgRew: 70.72 | StdRew: 8.73 | Time/Step: 0.0001


Training Episode:   8%|▊         | 4/50 [00:00<00:07,  6.46it/s]

[TRAIN][Step 6089] AvgLen: 51.08 | AvgRew: 68.38 | StdRew: 7.98 | Time/Step: 0.0001
[TRAIN][Step 8098] AvgLen: 51.51 | AvgRew: 66.96 | StdRew: 8.91 | Time/Step: 0.0001


Training Episode:  12%|█▏        | 6/50 [00:00<00:06,  6.48it/s]

...

[TRAIN][Step 95244] AvgLen: 52.33 | AvgRew: 65.52 | StdRew: 8.37 | Time/Step: 0.0001
[TRAIN][Step 97268] AvgLen: 51.90 | AvgRew: 68.51 | StdRew: 7.17 | Time/Step: 0.0001


Training Episode: 100%|██████████| 50/50 [00:07<00:00,  6.56it/s]

[TRAIN][Step 99269] AvgLen: 52.66 | AvgRew: 69.00 | StdRew: 8.72 | Time/Step: 0.0001
[TRAIN][Step 101316] AvgLen: 52.49 | AvgRew: 70.18 | StdRew: 9.23 | Time/Step: 0.0001
Training plot path:  ./logs/UCB_bandit/train/plot_data.json



Evaluation Episode: 100%|██████████| 5/5 [00:00<00:00, 273.99it/s]

[EVAL] CurrentAgent | Episode 1 | Reward: 73.49 | Length: 61
[EVAL] CurrentAgent | Episode 2 | Reward: 69.25 | Length: 46
[EVAL] CurrentAgent | Episode 3 | Reward: 68.00 | Length: 39
[EVAL] CurrentAgent | Episode 4 | Reward: 60.00 | Length: 41
[EVAL] CurrentAgent | Episode 5 | Reward: 64.00 | Length: 51
[EVAL][Step 0] AvgLen: 47.60 | AvgRew: 66.95 | StdRew: 4.61 | Time/Step: 0.0000
Plot path:  ./logs/UCB_bandit/train/eval_plot_data.json


In [7]:
tmp_base_dir = './logs/Contextual_bandit'
from contextual_bandit import ContextualBanditAgent

def create_contextual_bandit_agent(env, **kwargs):
    n_arms = env.action_space.nvec[0]  # Use .nvec[0] for MultiDiscrete
    return ContextualBanditAgent(n_arms=n_arms, epsilon=0.1)

train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)

runner_cb = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_contextual_bandit_agent,
    env=interest_evolution.create_environment(env_config),
)

runner_cb.run_training(max_training_steps=2000, num_iterations=50)
runner_cb.run_evaluation(max_eval_episodes=5)

Training Episode:   4%|▍         | 2/50 [00:00<00:07,  6.57it/s]

[TRAIN][Step 2033] AvgLen: 48.40 | AvgRew: 67.16 | StdRew: 8.50 | Time/Step: 0.0001
[TRAIN][Step 4044] AvgLen: 50.27 | AvgRew: 69.61 | StdRew: 8.69 | Time/Step: 0.0001


Training Episode:   8%|▊         | 4/50 [00:00<00:06,  6.77it/s]

[TRAIN][Step 6097] AvgLen: 50.07 | AvgRew: 66.97 | StdRew: 7.62 | Time/Step: 0.0001
[TRAIN][Step 8127] AvgLen: 49.51 | AvgRew: 68.59 | StdRew: 8.45 | Time/Step: 0.0001


Training Episode:  12%|█▏        | 6/50 [00:00<00:06,  6.92it/s]

...

[EVAL][Step 99240] AvgLen: 56.40 | AvgRew: 70.95 | StdRew: 10.38 | Time/Step: 0.0000


Evaluation Episode:   0%|          | 0/5 [00:00<?, ?it/s]

[EVAL] ckpt_49.pkl | Episode 1 | Reward: 76.00 | Length: 72
[EVAL] ckpt_49.pkl | Episode 2 | Reward: 68.00 | Length: 57
[EVAL] ckpt_49.pkl | Episode 3 | Reward: 72.00 | Length: 39
[EVAL] ckpt_49.pkl | Episode 4 | Reward: 92.00 | Length: 55


Evaluation Episode: 100%|██████████| 5/5 [00:00<00:00, 257.21it/s]

[EVAL] ckpt_49.pkl | Episode 5 | Reward: 62.17 | Length: 42
[EVAL][Step 101274] AvgLen: 53.00 | AvgRew: 74.03 | StdRew: 10.08 | Time/Step: 0.0000
Plot path:  ./logs/Contextual_bandit/train/eval_plot_data.json


In [14]:
from naive_dqn import NaiveDQNAgent
from gymnasium import spaces
tmp_base_dir = './logs/naive_dqn'

def create_naive_dqn_agent(env, **kwargs):
    doc_obs_space = env.observation_space['doc']

    if isinstance(doc_obs_space, spaces.Dict):
        # 每个文档的 shape 应该一致，取第一个文档的 shape[0]
        first_key = list(doc_obs_space.spaces.keys())[0]
        per_doc_dim = doc_obs_space.spaces[first_key].shape[0]
        num_docs = len(doc_obs_space.spaces)
        obs_dim = per_doc_dim * num_docs  # e.g., 7 * 10 = 70
    else:
        raise ValueError("Unsupported doc observation space:", doc_obs_space)

    if isinstance(env.action_space, spaces.Discrete):
        n_actions = env.action_space.n
    elif isinstance(env.action_space, spaces.MultiDiscrete):
        n_actions = env.action_space.nvec[0]
    else:
        raise ValueError("Unsupported action space type")

    print(f"obs_dim: {obs_dim}, n_actions: {n_actions}")
    return NaiveDQNAgent(
        obs_dim=obs_dim,
        n_actions=n_actions,
        epsilon=kwargs.get('epsilon', 0.1),
        gamma=kwargs.get('gamma', 0.99),
        lr=kwargs.get('lr', 1e-3)
    )

train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)
runner_naive_dqn = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_naive_dqn_agent,
    env=interest_evolution.create_environment(env_config),
)

obs_dim: 70, n_actions: 10


In [22]:
from naive_dqn import NaiveDQNAgent
from gymnasium import spaces
tmp_base_dir = './logs/naive_dqn'

def create_naive_dqn_agent(env, **kwargs):
    doc_obs_space = env.observation_space['doc']

    if isinstance(doc_obs_space, spaces.Dict):
        # 每个文档的 shape 应该一致，取第一个文档的 shape[0]
        first_key = list(doc_obs_space.spaces.keys())[0]
        per_doc_dim = doc_obs_space.spaces[first_key].shape[0]
        num_docs = len(doc_obs_space.spaces)
        obs_dim = per_doc_dim * num_docs  # e.g., 7 * 10 = 70
    else:
        raise ValueError("Unsupported doc observation space:", doc_obs_space)

    if isinstance(env.action_space, spaces.Discrete):
        n_actions = env.action_space.n
    elif isinstance(env.action_space, spaces.MultiDiscrete):
        n_actions = env.action_space.nvec[0]
    else:
        raise ValueError("Unsupported action space type")

    print(f"obs_dim: {obs_dim}, n_actions: {n_actions}")
    return NaiveDQNAgent(
        obs_dim=obs_dim,
        n_actions=n_actions,
        epsilon=kwargs.get('epsilon', 0.1),
        gamma=kwargs.get('gamma', 0.99),
        lr=kwargs.get('lr', 1e-3)
    )

train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)
runner_naive_dqn = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_naive_dqn_agent,
    env=interest_evolution.create_environment(env_config),
)

obs_dim: 70, n_actions: 10


In [26]:
from naive_dqn import NaiveDQNAgent
from gymnasium import spaces
tmp_base_dir = './logs/naive_dqn'

def create_naive_dqn_agent(env, **kwargs):
    doc_obs_space = env.observation_space['doc']

    if isinstance(doc_obs_space, spaces.Dict):
        # 每个文档的 shape 应该一致，取第一个文档的 shape[0]
        first_key = list(doc_obs_space.spaces.keys())[0]
        per_doc_dim = doc_obs_space.spaces[first_key].shape[0]
        num_docs = len(doc_obs_space.spaces)
        obs_dim = per_doc_dim * num_docs  # e.g., 7 * 10 = 70
    else:
        raise ValueError("Unsupported doc observation space:", doc_obs_space)

    if isinstance(env.action_space, spaces.Discrete):
        n_actions = env.action_space.n
    elif isinstance(env.action_space, spaces.MultiDiscrete):
        n_actions = env.action_space.nvec[0]
    else:
        raise ValueError("Unsupported action space type")

    print(f"obs_dim: {obs_dim}, n_actions: {n_actions}")
    return NaiveDQNAgent(
        obs_dim=obs_dim,
        n_actions=n_actions,
        epsilon=kwargs.get('epsilon', 0.1),
        gamma=kwargs.get('gamma', 0.99),
        lr=kwargs.get('lr', 1e-3)
    )

train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)

In [27]:
runner_naive_dqn = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_naive_dqn_agent,
    env=interest_evolution.create_environment(env_config),
)

obs_dim: 70, n_actions: 10


In [28]:
runner_naive_dqn.run_training(max_training_steps=2000, num_iterations=50)
runner_naive_dqn.run_evaluation(max_eval_episodes=5)

Training Episode:   0%|          | 0/50 [00:00<?, ?it/s]/projectnb/vkolagrp/yiliu/hrandomw/project_2/recsim_2025/naive_dqn.py:84: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  s, a, r, s_next, done = zip(*batch)
Training Episode:   2%|▏         | 1/50 [00:05<04:25,  5.41s/it]

[TRAIN][Step 2019] AvgLen: 49.24 | AvgRew: 67.77 | StdRew: 8.12 | Time/Step: 0.0027


Training Episode:   4%|▍         | 2/50 [00:08<03:00,  3.76s/it]

[TRAIN][Step 4072] AvgLen: 54.03 | AvgRew: 74.49 | StdRew: 10.06 | Time/Step: 0.0013


Training Episode:   6%|▌         | 3/50 [00:10<02:30,  3.19s/it]

...

[EVAL] ckpt_49.pkl | Episode 2 | Reward: 77.35 | Length: 73


Evaluation Episode:  80%|████████  | 4/5 [00:00<00:00, 10.94it/s]

[EVAL] ckpt_49.pkl | Episode 3 | Reward: 64.54 | Length: 85
[EVAL] ckpt_49.pkl | Episode 4 | Reward: 68.00 | Length: 61


Evaluation Episode: 100%|██████████| 5/5 [00:00<00:00, 10.31it/s]

[EVAL] ckpt_49.pkl | Episode 5 | Reward: 102.30 | Length: 94
[EVAL][Step 101417] AvgLen: 75.20 | AvgRew: 78.44 | StdRew: 13.23 | Time/Step: 0.0000
Plot path:  ./logs/naive_dqn/train/eval_plot_data.json


In [31]:
# Double DQN
from double_dqn import DoubleDQNAgent
tmp_base_dir = './logs/double_dqn'

def create_double_dqn_agent(env, **kwargs):
    doc_obs_space = env.observation_space['doc']

    if isinstance(doc_obs_space, spaces.Dict):
        first_key = list(doc_obs_space.spaces.keys())[0]
        per_doc_dim = doc_obs_space.spaces[first_key].shape[0]
        num_docs = len(doc_obs_space.spaces)
        obs_dim = per_doc_dim * num_docs
    else:
        raise ValueError("Unsupported doc observation space:", doc_obs_space)

    if isinstance(env.action_space, spaces.Discrete):
        n_actions = env.action_space.n
    elif isinstance(env.action_space, spaces.MultiDiscrete):
        n_actions = env.action_space.nvec[0]
    else:
        raise ValueError("Unsupported action space type")

    print(f"[DoubleDQN] obs_dim: {obs_dim}, n_actions: {n_actions}")
    return DoubleDQNAgent(
        obs_dim=obs_dim,
        n_actions=n_actions,
        epsilon=kwargs.get('epsilon', 0.1),
        gamma=kwargs.get('gamma', 0.99),
        lr=kwargs.get('lr', 1e-3),
        tau=kwargs.get('tau', 0.005)  # soft update rate
    )
train_log_dir = os.path.join(tmp_base_dir, 'train')
eval_log_dir = os.path.join(tmp_base_dir, 'eval')

for log_dir in [train_log_dir, eval_log_dir]:
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

Path(tmp_base_dir).mkdir(parents=True, exist_ok=True)
runner_double_dqn = Runner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_double_dqn_agent,
    env=interest_evolution.create_environment(env_config),
)

[DoubleDQN] obs_dim: 70, n_actions: 10


In [32]:
runner_double_dqn.run_training(max_training_steps=2000, num_iterations=50)
runner_double_dqn.run_evaluation(max_eval_episodes=5)

Training Episode:   2%|▏         | 1/50 [00:02<02:21,  2.88s/it]

[TRAIN][Step 2040] AvgLen: 46.36 | AvgRew: 67.99 | StdRew: 6.52 | Time/Step: 0.0014


Training Episode:   4%|▍         | 2/50 [00:05<02:18,  2.89s/it]

[TRAIN][Step 4072] AvgLen: 50.80 | AvgRew: 68.95 | StdRew: 7.43 | Time/Step: 0.0014


Training Episode:   6%|▌         | 3/50 [00:08<02:16,  2.90s/it]

...

[EVAL] ckpt_49.pkl | Episode 2 | Reward: 104.00 | Length: 42
[EVAL] ckpt_49.pkl | Episode 3 | Reward: 76.00 | Length: 39


Evaluation Episode:  80%|████████  | 4/5 [00:00<00:00, 13.46it/s]

[EVAL] ckpt_49.pkl | Episode 4 | Reward: 88.00 | Length: 69


Evaluation Episode: 100%|██████████| 5/5 [00:00<00:00, 13.60it/s]

[EVAL] ckpt_49.pkl | Episode 5 | Reward: 70.14 | Length: 51
[EVAL][Step 101568] AvgLen: 50.60 | AvgRew: 86.51 | StdRew: 12.24 | Time/Step: 0.0000
Plot path:  ./logs/double_dqn/train/eval_plot_data.json
